# Africas Talking-Mozilla Common Voice Hackathon

This is the main notebook used in model development. For experimentation use notebooks in the `misc_notebooks` folder

## 1. Problem Statement

This project is originally a submission to the Africa's Talking x Mozilla Common Voice Hackathon, from October 13th through November

### 1.1 Objectives
>> The main objective is to build a deep learning model that is capable of inferring `text` having been conditioned on `voice` sequences.

The model is expected to achieve best performance on the selected evaluation metrics:
* Character Error Rate
* Word Error Rate
* Phone Error Rate
* Loss

## 2. Data Loading

The data used is sourced from Mozilla's site [here](https://commonvoice.mozilla.org/en/datasets)

In [1]:
# importations
import pandas as pd
import numpy as np
import os
from scripts import coqui_data_prepper
# import scripts/to_csv.py as to_csv


In [2]:
# read `tsv` files
test_df = pd.read_csv('sw/test.tsv', delimiter='\t')
train_df = pd.read_csv('sw/train.tsv', delimiter='\t')
# reported_df = pd.read_csv('sw/reported.tsv', engine='c', delimiter='\t') # causing EOF error in reading, possibly corrupt or inconsistent
# others_df = pd.read_csv('sw/other.tsv', delimiter='\t') # causing EOF error in reading, possibly corrupt or inconsistent
invalidated_df = pd.read_csv('sw/invalidated.tsv', delimiter='\t')
dev_df = pd.read_csv('sw/dev.tsv', delimiter='\t')
durations_df = pd.read_csv('sw/clip_durations.tsv', delimiter='\t')

## 3. EDA

Let's inspect some of these DataFrames to get our bearing

In [3]:
#train_df
train_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,common_voice_sw_37083656.mp3,yeyote yule atakaepatikana akiandamana katika ...,2,0,twenties,female,NaN,NaN,sw,NaN
1,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,common_voice_sw_37085166.mp3,kwa ujumla kwenye kwenye sehemu za wilaya ama ...,3,1,twenties,female,NaN,NaN,sw,NaN
2,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,common_voice_sw_37085182.mp3,Hawai ni kuzuri,2,0,twenties,female,NaN,NaN,sw,NaN
3,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,common_voice_sw_37085525.mp3,na kuhatarisha mifumo mizima ya uhai katika se...,13,3,twenties,female,NaN,NaN,sw,NaN
4,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,common_voice_sw_37085535.mp3,kundi zima lililosababisha mauaji lilitakiwa k...,2,1,twenties,female,NaN,NaN,sw,NaN


In [4]:
# test_df
test_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,011544b78c18417a5869b8e70ebc7675c7eb3b517754b8...,common_voice_sw_37190902.mp3,Tuma pesa sahii.,2,0,NaN,NaN,NaN,NaN,sw,NaN
1,0133d8ddf5c1a3c678fde017e0b07d2835bfd707d5b3ec...,common_voice_sw_31428161.mp3,wachambuzi wa soka wanamtaja Messi kama nyota ...,2,0,twenties,female,NaN,NaN,sw,NaN
2,01c95772efd3fbe4a1122206c7474c77ed6591c8c9fb00...,common_voice_sw_30317714.mp3,romario aliingia kwenye orodha ya wachezaji wa...,2,1,NaN,NaN,NaN,NaN,sw,NaN
3,023711185d4404ff398c2697f2e72868d1ecf69a92b581...,common_voice_sw_32116997.mp3,Sote twesangaa twelipomuona mwalimu Ali apika,2,1,twenties,male,NaN,NaN,sw,NaN
4,0244639ffd7ec755a01b21ea204735ca3c44443e9cf46c...,common_voice_sw_29002392.mp3,Inajulikana kama shina la Warangi.,2,0,NaN,NaN,NaN,NaN,sw,NaN


In [5]:
# dev_df
dev_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,common_voice_sw_30628088.mp3,Mbali na kuwa afisa wa serikali Jokate pia ni ...,2,0,twenties,male,NaN,NaN,sw,NaN
1,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,common_voice_sw_30628121.mp3,Kukosa pesa ni hatari sana,3,1,twenties,male,NaN,NaN,sw,NaN
2,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,common_voice_sw_30628126.mp3,Jina Karagwe linatokana na kilima kinachopatik...,2,0,twenties,male,NaN,NaN,sw,NaN
3,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,common_voice_sw_30628160.mp3,sokwe wanaopatikana mahale ni aina adimu zaidi...,2,0,twenties,male,NaN,NaN,sw,NaN
4,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,common_voice_sw_30628161.mp3,mamba wa mto naili wana sumu kali inayoozesha ...,3,0,twenties,male,NaN,NaN,sw,NaN


We need to change the path to reflect the relative path in this directory. We can then create new dfs to be used to train on coqui_stt

In [6]:
def attach_prefix(df, prefix):
  """Attaches a prefix to all entries in the column 'path' of a dataframe.

  Args:
    df: A Pandas dataframe with the column 'path'.
    prefix: The prefix to attach to the entries in the column 'path'.

  Returns:
    A Pandas dataframe with the column 'path' updated to include the prefix.
  """

  df["path"] = df["path"].apply(lambda x: prefix + x)
  return df



In [7]:
# Create a list of Pandas DataFrames
df_list = [train_df, dev_df, test_df]

# iterate over the dfs and attatch directory prefix on each
for df, df_name in zip(df_list, ["train", "dev", "test"]):
    df = df

    prefixed_df = attach_prefix(df, "sw/clips/")

In [8]:
train_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,sw/clips/common_voice_sw_37083656.mp3,yeyote yule atakaepatikana akiandamana katika ...,2,0,twenties,female,NaN,NaN,sw,NaN
1,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,sw/clips/common_voice_sw_37085166.mp3,kwa ujumla kwenye kwenye sehemu za wilaya ama ...,3,1,twenties,female,NaN,NaN,sw,NaN
2,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,sw/clips/common_voice_sw_37085182.mp3,Hawai ni kuzuri,2,0,twenties,female,NaN,NaN,sw,NaN
3,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,sw/clips/common_voice_sw_37085525.mp3,na kuhatarisha mifumo mizima ya uhai katika se...,13,3,twenties,female,NaN,NaN,sw,NaN
4,70c2b9896ca6150002ef6e888a3f7788e49e61aef85ec8...,sw/clips/common_voice_sw_37085535.mp3,kundi zima lililosababisha mauaji lilitakiwa k...,2,1,twenties,female,NaN,NaN,sw,NaN


In [9]:
dev_df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,sw/clips/common_voice_sw_30628088.mp3,Mbali na kuwa afisa wa serikali Jokate pia ni ...,2,0,twenties,male,NaN,NaN,sw,NaN
1,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,sw/clips/common_voice_sw_30628121.mp3,Kukosa pesa ni hatari sana,3,1,twenties,male,NaN,NaN,sw,NaN
2,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,sw/clips/common_voice_sw_30628126.mp3,Jina Karagwe linatokana na kilima kinachopatik...,2,0,twenties,male,NaN,NaN,sw,NaN
3,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,sw/clips/common_voice_sw_30628160.mp3,sokwe wanaopatikana mahale ni aina adimu zaidi...,2,0,twenties,male,NaN,NaN,sw,NaN
4,8ed66acaaecf3d1ffd887ed5af76a220976a8b3dd1209f...,sw/clips/common_voice_sw_30628161.mp3,mamba wa mto naili wana sumu kali inayoozesha ...,3,0,twenties,male,NaN,NaN,sw,NaN


## 4. Preprocessing

Coqui-STT requires data to be in a specific format before training begins, i.e:
* wav_filename: PATH to the wav files ( not mp3; format not supported)
* wav_filesize: size of each wav clip in bytes
* transcript: the sentence column transcribed for each clip

**Note**: there are scripts to convert mp3 to wav and to do this preprocessing in the scripts module incuded in this project

In [14]:
from pydub import AudioSegment

def convert_mp3_to_wav(df, label):
  """Converts all MP3 files in the `path` column to WAV files and saves them to a new directory (test, train, or dev) in the same folder as the original MP3 files.

  Args:
    df: A Pandas DataFrame.
    label: The label for the new directory (test, train, or dev).

  Returns:
    A Pandas DataFrame with the `path` column updated to point to the new WAV files.
  """

  # Create a new directory for the WAV files.
  new_dir = os.path.join(df['path'].iloc[0].split('/')[0], label)
  os.makedirs(new_dir, exist_ok=True)

  # Iterate over the DataFrame and convert each MP3 file to WAV.
  for i in range(len(df)):
    mp3_path = df['path'].iloc[i]
    wav_path = os.path.join(new_dir, os.path.basename(mp3_path).replace('.mp3', '.wav'))

    # Convert the MP3 file to WAV.
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format='wav')

    # Update the `path` column to point to the new WAV file.
    df.loc[i, 'path'] = wav_path

  return df

>> **NOTE**: the 3 cells below need only be ran once and they may take some time to run as they are parsing a rather large amount of data

In [16]:
# convert train_df clips to .wav
train_df = convert_mp3_to_wav(train_df, 'train')

KeyboardInterrupt: 

In [18]:
# convert dev_df clips to .wav
tdev_df = convert_mp3_to_wav(dev_df, 'dev')

KeyboardInterrupt: 

In [19]:
# convert test_df clips to .wav
test_df = convert_mp3_to_wav(test_df, 'test')

KeyboardInterrupt: 

>> **END NOTE**: try commenting the cells above if they have already be ran

In [22]:
# format DFs for training (Repetitive), needs to be better implemented (DRY)
train_df = coqui_data_prepper.format_df(train_df)
dev_df = coqui_data_prepper.format_df(dev_df)
test_df = coqui_data_prepper.format_df(test_df)

KeyError: 'path'

In [21]:
train_df.head()

,wav_filename,wav_filesize,transcript
0,train/common_voice_sw_37083656.wav.wav,359468,yeyote yule atakaepatikana akiandamana katika ...
1,train/common_voice_sw_37085166.wav.wav,391724,kwa ujumla kwenye kwenye sehemu za wilaya ama ...
2,train/common_voice_sw_37085182.wav.wav,191276,Hawai ni kuzuri
3,train/common_voice_sw_37085525.wav.wav,391724,na kuhatarisha mifumo mizima ya uhai katika se...
4,train/common_voice_sw_37085535.wav.wav,391724,kundi zima lililosababisha mauaji lilitakiwa k...


In [12]:
test_df.head()

,wav_filename,wav_filesize,transcript
0,train/common_voice_sw_37190902.wav,21213,Tuma pesa sahii.
1,train/common_voice_sw_31428161.wav,47133,wachambuzi wa soka wanamtaja Messi kama nyota ...
2,train/common_voice_sw_30317714.wav,31149,romario aliingia kwenye orodha ya wachezaji wa...
3,train/common_voice_sw_32116997.wav,60093,Sote twesangaa twelipomuona mwalimu Ali apika
4,train/common_voice_sw_29002392.wav,27693,Inajulikana kama shina la Warangi.


In [13]:
# convert transcript to lower case characters
def to_lower(df, column_name):
  """Converts all characters in a column to lowercase.

  Args:
    df: A Pandas DataFrame.
    column_name: The name of the column to convert to lowercase.

  Returns:
    A Pandas DataFrame with the column `column_name` converted to lowercase.
  """

  df[column_name] = df[column_name].str.lower()
  return df

In [14]:
train_df = to_lower(train_df, 'transcript')
dev_df = to_lower(dev_df, 'transcript')
test_df = to_lower(test_df, 'transcript')

In [15]:
train_df.head()

,wav_filename,wav_filesize,transcript
0,train/common_voice_sw_37083656.wav,33741,yeyote yule atakaepatikana akiandamana katika ...
1,train/common_voice_sw_37085166.wav,36765,kwa ujumla kwenye kwenye sehemu za wilaya ama ...
2,train/common_voice_sw_37085182.wav,17973,hawai ni kuzuri
3,train/common_voice_sw_37085525.wav,36765,na kuhatarisha mifumo mizima ya uhai katika se...
4,train/common_voice_sw_37085535.wav,36765,kundi zima lililosababisha mauaji lilitakiwa k...


Need to convert the *train*, *dev* and *test* tsc files to csv to work with `coqui`

In [16]:
# Create a list of Pandas DataFrames
df_list = [train_df, dev_df, test_df]

# Create the sw/ directory if it does not exist
csv_directory_path = "sw/"
if not os.path.exists(csv_directory_path):
    os.makedirs(csv_directory_path)

# Iterate over the list of DataFrames and save each DataFrame to a CSV file
for df, df_name in zip(df_list, ["train", "dev", "test"]):

    # Join the DataFrame name with the CSV directory path
    csv_file_path = os.path.join(csv_directory_path, df_name + ".csv")

    # Save the DataFrame to the CSV file
    df.to_csv(csv_file_path, index=False)

In [17]:
# Implement wave tokenizer to obtain feature table


In [18]:
# detect alphabet
import chardet

def detect_characters_in_dataframes(train_df, dev_df, test_df, alphabet_file_path):
  """Detects all the characters in the column 'transcript' for dataframes 'train_df, dev_df and test_df' and saves them to a file.

  Args:
    train_df: A Pandas dataframe containing the training data.
    dev_df: A Pandas dataframe containing the development data.
    test_df: A Pandas dataframe containing the test data.
    alphabet_file_path: The path to the file to save the alphabet to.
  """

  alphabet = set()

  for df in [train_df, dev_df, test_df]:
    transcripts = df["transcript"]

    for transcript in transcripts:
      characters = chardet.detect(transcript.encode())["encoding"]

      for character in characters:
        alphabet.add(character)

  with open(alphabet_file_path, "w") as alphabet_file:
    for character in alphabet:
      alphabet_file.write(character + "\n")

In [19]:
# alphabet_file_path = "sw/alphabet.txt"

# detect_characters_in_dataframes(train_df, dev_df, test_df, alphabet_file_path)

## 5. Modelling

### INITIALIZE DEFAULT HYPERPARAMETERS (Coqui-stt)

In [2]:
from coqui_stt_training.util.config import initialize_globals_from_args

initialize_globals_from_args(
    checkpoint_dir="ckpt_dir",
    train_files=["sw/train.csv"],
    dev_files=["sw/dev.csv"],
    test_files=["sw/test.csv"],
    load_train="init",
    n_hidden=200,
    epochs=10,
    train_batch_size=1,
    dev_batch_size=1,
    test_batch_size= 1,
)

2023-10-19 02:38:39.150351: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-10-19 02:38:39.188002: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2023-10-19 02:38:39.190704: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x40550b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-19 02:38:39.190782: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
I --alphabet_config_path not specified, but found an alphabet file alongside specified checkpoint (ckpt_dir/alphabet.txt). Will use this alphabet file for this run.


In [3]:
! python -m coqui_stt_training.train --train_files sw/train.csv --dev_files sw/dev.csv --test_files sw/test.csv --checkpoint_dir ckpt_dir

I --alphabet_config_path not specified, but found an alphabet file alongside specified checkpoint (ckpt_dir/alphabet.txt). Will use this alphabet file for this run.
W WARNING: --n_hidden value (2048) is different from value found in checkpoint (200).
W WARNING: This would result in an error when loading the checkpoint, so n_hidden has been overriden with the checkpoint value.
I Performing dummy training to check for memory problems.
I If the following process crashes, you likely have batch sizes that are too big for your available system memory (or GPU memory).
I Could not find best validating checkpoint.
I Could not find most recent checkpoint.
I Initializing all variables.
I STARTING Optimization
Epoch 0 |   Training | Elapsed Time: 0:00:05 | Steps: 3 | Loss: 946.650411     
Epoch 0 | Validation | Elapsed Time: 0:00:00 | Steps: 1 | Loss: 375.970184 | Dataset: sw/dev.csvTraceback (most recent call last):
  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-

In [9]:
# Kick off training job; configures CUDA to only use one GPU
from coqui_stt_training.train import train

# use maximum one GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

train()

I Performing dummy training to check for memory problems.
I If the following process crashes, you likely have batch sizes that are too big for your available system memory (or GPU memory).
I Initializing all variables.
I STARTING Optimization
Epoch 0 |   Training | Elapsed Time: 0:00:00 | Steps: 0 | Loss: 0.000000
Epoch 0 |   Training | Elapsed Time: 0:00:01 | Steps: 0 | Loss: 0.000000
Epoch 0 | Validation | Elapsed Time: 0:00:00 | Steps: 0 | Loss: 0.000000 | Dataset: sw/dev.csv


2023-10-18 09:35:55.794194: W tensorflow/core/framework/op_kernel.cc:1639] Invalid argument: ValueError: Alphabet cannot encode transcript "yoweri museveni alikuwako kati ya wanamgambo waliosaidiana na watanzania" while processing sample "sw/train/common_voice_sw_30556325.wav", check that your alphabet contains all characters in the training corpus. Missing characters are: [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '].
Traceback (most recent call last):

  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", line 235, in __call__
    ret = func(*args)

  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-packages/tensorflow_core/python/data/ops/dataset_ops.py", line 594, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-packages/coqui_stt_training/util/feeding.py", line 188,

UnknownError: FileNotFoundError: [Errno 2] No such file or directory: 'sw/train/common_voice_sw_35169173.wav'
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-packages/coqui_stt_training/util/augmentations.py", line 192, in _load_and_augment_sample
    realized_sample = unpack_maybe(sample)
  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-packages/coqui_stt_training/util/sample_collections.py", line 93, in unpack_maybe
    realized_sample = sample.unpack()
  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/python3.7/site-packages/coqui_stt_training/util/sample_collections.py", line 80, in unpack
    with open_remote(self.filename, "rb") as audio_file:
  File "/home/josh/Desktop/proj/mcv-stt-hackathon/mcv-stt-env/lib/pyt

### Try Transfer Learning

In [ ]:
# Import pretrained model


# END